In [9]:
from gen_data_and_simulate_drift import DriftIntensity, DriftSimulator, GenerateFakeData
from training_script import Classifier

In [10]:
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import OrdinalEncoder

In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
DATA_PATH = '../datasets/'
MODEL_PATH = '../models/'

### Exploring the generator class

In [13]:
pokemon_test_data='bank_num.csv'
df = pd.read_csv(DATA_PATH + pokemon_test_data )

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11162 entries, 0 to 11161
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        11162 non-null  int64 
 1   job        11162 non-null  int64 
 2   marital    11162 non-null  int64 
 3   education  11162 non-null  int64 
 4   default    11162 non-null  object
 5   balance    11162 non-null  int64 
 6   housing    11162 non-null  int64 
 7   loan       11162 non-null  int64 
 8   contact    11162 non-null  int64 
 9   day        11162 non-null  int64 
 10  month      11162 non-null  int64 
 11  duration   11162 non-null  int64 
 12  campaign   11162 non-null  int64 
 13  pdays      11162 non-null  int64 
 14  previous   11162 non-null  int64 
 15  poutcome   11162 non-null  int64 
 16  deposit    11162 non-null  int64 
dtypes: int64(16), object(1)
memory usage: 1.4+ MB


In [15]:
SAMPLE_SIZE=1000
target = 'default'
genertor_fake_data = GenerateFakeData(path_ref_data=DATA_PATH+'bank_num.csv', sample_size=SAMPLE_SIZE, target=target, model_name = 'bank_generator.pkl')

default


In [16]:
genertor_fake_data.df_ref.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11162 entries, 0 to 11161
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        11162 non-null  int64 
 1   job        11162 non-null  int64 
 2   marital    11162 non-null  int64 
 3   education  11162 non-null  int64 
 4   balance    11162 non-null  int64 
 5   housing    11162 non-null  int64 
 6   loan       11162 non-null  int64 
 7   contact    11162 non-null  int64 
 8   day        11162 non-null  int64 
 9   month      11162 non-null  int64 
 10  duration   11162 non-null  int64 
 11  campaign   11162 non-null  int64 
 12  pdays      11162 non-null  int64 
 13  previous   11162 non-null  int64 
 14  poutcome   11162 non-null  int64 
 15  deposit    11162 non-null  int64 
 16  default    11162 non-null  object
dtypes: int64(16), object(1)
memory usage: 1.4+ MB


In [17]:
genertor_fake_data.df_samples.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 499
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        1000 non-null   int64 
 1   job        1000 non-null   int64 
 2   marital    1000 non-null   int64 
 3   education  1000 non-null   int64 
 4   balance    1000 non-null   int64 
 5   housing    1000 non-null   int64 
 6   loan       1000 non-null   int64 
 7   contact    1000 non-null   int64 
 8   day        1000 non-null   int64 
 9   month      1000 non-null   int64 
 10  duration   1000 non-null   int64 
 11  campaign   1000 non-null   int64 
 12  pdays      1000 non-null   int64 
 13  previous   1000 non-null   int64 
 14  poutcome   1000 non-null   int64 
 15  deposit    1000 non-null   int64 
 16  default    1000 non-null   object
dtypes: int64(16), object(1)
memory usage: 140.6+ KB


In [18]:
genertor_fake_data.df_samples.head()

,age,job,marital,education,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit,default
0,33,6,2,1,1850,1,0,0,31,4,1295,0,327,2,0,1,no
1,43,9,1,1,867,0,0,0,12,5,223,0,-5,0,2,0,no
2,32,10,1,1,725,0,0,0,12,9,181,2,-7,0,3,0,no
3,47,4,0,1,706,1,0,0,18,8,320,1,14,0,2,0,no
4,55,0,0,0,361,0,0,0,29,-1,592,3,123,0,2,0,no


In [19]:
sampled_data = genertor_fake_data.get_dataclass_sampling()

In [20]:
sampled_data.train_data.describe()

,age,job,marital,education,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
count,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.00000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000
mean,40.263750,3.903750,0.736250,0.803750,818.717500,0.253750,0.140000,0.387500,16.30000,4.873750,338.348750,2.316250,29.796250,0.385000,2.217500,0.182500
std,10.985674,3.289455,0.712264,0.811517,2469.401219,0.435428,0.347204,0.707439,8.78829,3.018559,301.615101,3.382016,93.902872,1.458401,0.853583,0.386498
min,18.000000,-2.000000,0.000000,0.000000,-6733.000000,0.000000,0.000000,-1.000000,-2.00000,-1.000000,-116.000000,-2.000000,-82.000000,-1.000000,0.000000,0.000000
25%,32.000000,1.000000,0.000000,0.000000,-205.500000,0.000000,0.000000,0.000000,9.00000,3.000000,131.750000,1.000000,-12.000000,0.000000,2.000000,0.000000
50%,39.000000,3.000000,1.000000,1.000000,209.500000,0.000000,0.000000,0.000000,16.00000,5.000000,249.000000,1.000000,3.000000,0.000000,2.000000,0.000000
75%,47.000000,7.000000,1.000000,1.000000,950.000000,1.000000,0.000000,1.000000,24.00000,7.000000,467.000000,3.000000,27.000000,0.000000,3.000000,0.000000
max,84.000000,11.000000,2.000000,3.000000,31783.000000,1.000000,1.000000,3.000000,36.00000,11.000000,1958.000000,28.000000,765.000000,14.000000,3.000000,1.000000


In [21]:
sampled_data.test_data.describe()

,age,job,marital,education,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,40.710000,4.205000,0.805000,0.725000,1160.660000,0.260000,0.085000,0.555000,16.490000,4.985000,343.600000,2.360000,36.220000,0.340000,2.150000,0.215000
std,11.570448,3.435402,0.706734,0.743058,4534.226005,0.439735,0.279582,0.754867,9.056489,2.882066,303.878265,3.122362,90.681183,1.118326,0.855076,0.411853
min,17.000000,-2.000000,0.000000,0.000000,-1266.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,-75.000000,-2.000000,-70.000000,0.000000,0.000000,0.000000
25%,32.000000,1.000000,0.000000,0.000000,-204.000000,0.000000,0.000000,0.000000,9.000000,4.000000,111.500000,0.000000,-8.250000,0.000000,2.000000,0.000000
50%,41.000000,4.000000,1.000000,1.000000,235.000000,0.000000,0.000000,0.000000,17.000000,5.000000,250.000000,2.000000,4.000000,0.000000,2.000000,0.000000
75%,48.250000,8.000000,1.000000,1.000000,966.500000,1.000000,0.000000,1.000000,24.000000,7.000000,490.500000,3.000000,32.500000,0.000000,3.000000,0.000000
max,80.000000,11.000000,2.000000,3.000000,56968.000000,1.000000,1.000000,2.000000,34.000000,11.000000,1755.000000,24.000000,452.000000,7.000000,3.000000,1.000000


In [22]:
sampled_data.list_num_col

['age',
 'job',
 'marital',
 'education',
 'balance',
 'housing',
 'loan',
 'contact',
 'day',
 'month',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'deposit']

In [24]:
pok_classifier = Classifier(df_train=sampled_data.train_data,
                 num_features=sampled_data.list_num_col,
                 cat_features=None,
                 target=target,
                pkl_file_path=MODEL_PATH + f'class_{target}_model.pkl',
                iterations = 200, depth = None, learning_rate = 0.03)

In [25]:
pok_classifier.train()

CatBoostError: Bad value for num_feature[non_default_doc_idx=0,feature_idx=16]="yes": Cannot convert 'b'yes'' to float

In [317]:
pok_classifier.serialize()

Error occurred while pickling: Can't pickle <class 'training_script.Classifier'>: it's not the same object as training_script.Classifier


In [323]:
prediction = pok_classifier.predict(sampled_data.train_data.drop(target, axis = 1))
#prediction = [1 if e=='True' else 0 for e in prediction]
prediction[:4]

array(['no', 'yes', 'no', 'yes'], dtype=object)

In [324]:
(prediction==sampled_data.train_data[target].values).mean()

0.90125

In [325]:
prediction = pok_classifier.predict(sampled_data.test_data.drop(target, axis = 1))
#prediction = [1 if e=='True' else 0 for e in prediction]
(prediction==sampled_data.test_data[target].values).mean()

0.765

### Analyzing the drift

In [349]:
from gen_data_and_simulate_drift import get_shift_coef

In [363]:
drift_sim_info = DriftSimulator(sampled_data, nb_cols_to_drift=1)

number of columns to drift is : 1
select random column to drift ...
Drifting column job


In [365]:
df_test_drifted = drift_sim_info.get_test_data_drifted()

In [367]:
df_test_drifted.job.describe()

count    200.000000
mean       6.265000
std        4.735706
min       -4.000000
25%        3.000000
50%        6.000000
75%        9.250000
max       22.000000
Name: job, dtype: float64

In [369]:
prediction = pok_classifier.predict(df_test_drifted.drop(target, axis = 1))
#prediction = [1 if e=='True' else 0 for e in prediction]
(prediction==sampled_data.test_data[target].values).mean()

0.74

In [368]:
sampled_data.test_data.job.describe()

count    200.000000
mean       4.000000
std        3.220537
min       -1.000000
25%        1.000000
50%        3.000000
75%        7.000000
max       11.000000
Name: job, dtype: float64

### Creating marketing dataset

In [66]:
import os

In [69]:
os.path.exists(DATA_PATH + 'bank.csv')

True

In [177]:
bank = pd.read_csv(DATA_PATH + 'bank.csv')

In [178]:
bank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11162 entries, 0 to 11161
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        11162 non-null  int64 
 1   job        11162 non-null  object
 2   marital    11162 non-null  object
 3   education  11162 non-null  object
 4   default    11162 non-null  object
 5   balance    11162 non-null  int64 
 6   housing    11162 non-null  object
 7   loan       11162 non-null  object
 8   contact    11162 non-null  object
 9   day        11162 non-null  int64 
 10  month      11162 non-null  object
 11  duration   11162 non-null  int64 
 12  campaign   11162 non-null  int64 
 13  pdays      11162 non-null  int64 
 14  previous   11162 non-null  int64 
 15  poutcome   11162 non-null  object
 16  deposit    11162 non-null  object
dtypes: int64(7), object(10)
memory usage: 1.4+ MB


In [179]:
cat_cols = bank.select_dtypes('O').columns
cat_cols = cat_cols.drop('default')
cat_cols

Index(['job', 'marital', 'education', 'housing', 'loan', 'contact', 'month',
       'poutcome', 'deposit'],
      dtype='object')

In [180]:
oe = OrdinalEncoder(dtype=np.int64, handle_unknown='use_encoded_value', unknown_value=-1)

In [181]:
bank[cat_cols].head()

,job,marital,education,housing,loan,contact,month,poutcome,deposit
0,admin.,married,secondary,yes,no,unknown,may,unknown,yes
1,admin.,married,secondary,no,no,unknown,may,unknown,yes
2,technician,married,secondary,yes,no,unknown,may,unknown,yes
3,services,married,secondary,yes,no,unknown,may,unknown,yes
4,admin.,married,tertiary,no,no,unknown,may,unknown,yes


In [182]:
bank[cat_cols] = oe.fit_transform(bank[cat_cols])

In [183]:
bank[cat_cols].head()

,job,marital,education,housing,loan,contact,month,poutcome,deposit
0,0,1,1,1,0,2,8,3,1
1,0,1,1,0,0,2,8,3,1
2,9,1,1,1,0,2,8,3,1
3,7,1,1,1,0,2,8,3,1
4,0,1,2,0,0,2,8,3,1


In [184]:
bank.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,0,1,1,no,2343,1,0,2,5,8,1042,1,-1,0,3,1
1,56,0,1,1,no,45,0,0,2,5,8,1467,1,-1,0,3,1
2,41,9,1,1,no,1270,1,0,2,5,8,1389,1,-1,0,3,1
3,55,7,1,1,no,2476,1,0,2,5,8,579,1,-1,0,3,1
4,54,0,1,2,no,184,0,0,2,5,8,673,2,-1,0,3,1


In [185]:
joblib.dump(oe, MODEL_PATH+ 'bank_ordinal_encoder.pkl')

['../models/bank_ordinal_encoder.pkl']

In [186]:
bank.to_csv(DATA_PATH + 'bank_num.csv', index = False)

In [187]:
bank.dtypes

age           int64
job           int64
marital       int64
education     int64
default      object
balance       int64
housing       int64
loan          int64
contact       int64
day           int64
month         int64
duration      int64
campaign      int64
pdays         int64
previous      int64
poutcome      int64
deposit       int64
dtype: object